In [1]:
import os

import numpy as np
import nibabel as nib
from scipy.ndimage.interpolation import zoom
import h5py

In [2]:
gt_root = '/home/yutongx/src_data/preprocessed_3d_128_xyt/'
pred_root = 'predictions/'
post_root = 'final_results/'

gt_path = os.path.join(gt_root, 'val')
pred_path = os.path.join(pred_root, 'checkpoint_4')
post_path = os.path.join(post_root, 'checkpoint_4')

In [3]:
def postprocessing_nii(gt_path, pred_path, post_path, order=0):
    gt_names = sorted(os.listdir(gt_path))
    pred_names = sorted(os.listdir(pred_path))

    for gt_name, pred_name in zip(gt_names, pred_names):
        file_gt = h5py.File(os.path.join(gt_path, gt_name),'r')
        gt_shape = file_gt['ori_shape'][:]
        gt_affine = file_gt['ori_affine'][:]
        file_gt.close()
        
        file_pred = h5py.File(os.path.join(pred_path, pred_name),'r')
        pred = file_pred['predictions'][0]
        file_pred.close()
        
        pred = np.moveaxis(pred, 0, -1)#to h*w*d
        scale = np.array(gt_shape) / np.array(pred.shape)
        pred_restored = (zoom(pred, scale, order=order, mode='nearest')).astype(np.float32)
  
        post_name = os.path.join(post_path, pred_name[:3] + '.nii.gz')
        nib.save(nib.Nifti1Image(pred_restored, gt_affine), post_name)

In [4]:
postprocessing_nii(gt_path, pred_path, post_path, order=0)